In [1]:
%matplotlib widget
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interact
from scipy import stats
import copy

from cvrp_simulation.scenario_generator import SampleDynamicBenchmark
from cvrp_simulation.simulator import CVRPSimulation


In [2]:
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

In [3]:
depot_position_rv = stats.uniform(loc=0, scale=1)
vehicle_position_rv = stats.uniform(loc=0, scale=1)
customer_positions_rv = stats.uniform(loc=0, scale=1)
customer_demands_rv = stats.randint(low=0, high=10)
customer_times_rv = stats.randint(low=0, high=5)
vehicle_velocity = 10
vrp_size = 20
initial_vehicle_capacity = vrp_size*2
benchmark_generator = SampleDynamicBenchmark(depot_position_rv=depot_position_rv, vehicle_position_rv=vehicle_position_rv, 
                                            vehicle_capacity=initial_vehicle_capacity, vehicle_velocity=vehicle_velocity,
                                            customer_positions_rv=customer_positions_rv, customer_demands_rv=customer_demands_rv,
                                            customer_times_rv=customer_times_rv,
                                            vrp_size=vrp_size)
seed = 5
sim = CVRPSimulation(max_customers=vrp_size, problem_generator=benchmark_generator)
sim.seed(seed)
color_map = get_cmap(10)
depot_color = color_map(0)
vehicle_color = color_map(1)

C:\Users\ros1tv\Anaconda3\envs\waferfab_test\lib\site-packages\gym-0.15.7-py3.7.egg\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
obs = sim.reset()
# in this case each time we choose action 0 since the available actions change each time
# the number of available customers changes
tot_reward = 0
done = False
all_states = [copy.deepcopy(sim)]
all_observations = [copy.deepcopy(obs)]
timeline = [0]
while not done:
    num_available_actions = sim.get_available_customers().size
    if num_available_actions>0:
        obs, reward, done, _ = sim.step(np.random.randint(0, num_available_actions))
    else:
        obs, reward, done, _ = sim.step(1)
    all_states.append(copy.deepcopy(sim))
    all_observations.append(copy.deepcopy(obs))
    timeline.append(int(np.ceil(sim.current_time)))
    tot_reward += reward
print(f"finished random run, total reward {tot_reward}")


finished random run, total reward -11.946657421815843


In [9]:
@interact
# for range (min, max, step) , this gives a slider
# for drop down give a list of the options
def plot_vehicle_routes(t=(0, len(timeline), 1)):
    plt.close()
    cur_sim = all_states[t]
    observation_space = all_observations[t]
    num_customers = cur_sim.current_state.customer_positions.shape[0]
    plt.scatter([], [], label='opened, demand<capacity',c='b')
    plt.scatter([], [], label='visited', c='g')
    plt.scatter([], [], label='opened, demand>capacity', c='m')
    plt.scatter([], [], label='closed', c='r')
    plt.scatter(cur_sim.current_state.depot_position[0], cur_sim.current_state.depot_position[1], marker='s', c=depot_color)
    
    for i_c in range(num_customers):
        customer_position = cur_sim.current_state.customer_positions[i_c, :]
        # opened customer
        plt.text(customer_position[0], customer_position[1],str(i_c))
        if cur_sim.current_state.customer_visited[i_c]:
            plt.scatter(customer_position[0], customer_position[1], marker='o', c='g')
        elif cur_sim.current_state.customer_times[i_c]<= timeline[t]:
            if cur_sim.current_state.customer_demands[i_c]<=cur_sim.current_state.current_vehicle_capacity:
                plt.scatter(customer_position[0], customer_position[1], marker='o', c='b')
            else:
                plt.scatter(customer_position[0], customer_position[1], marker='o', c='m')
                print(f"customer demand:{cur_sim.current_state.customer_demands[i_c]} , capacity:{cur_sim.current_state.current_vehicle_capacity}")
        elif cur_sim.current_state.customer_times[i_c]> timeline[t]:
            plt.scatter(customer_position[0], customer_position[1], marker='o', c='r')
        else:
            plt.scatter(customer_position[0], customer_position[1], marker='o', c='k')
    n_e = 0
    for i_c_o, obs_customer_position in enumerate(observation_space['customer_positions']):
        if obs_customer_position.all()!=0:
            plt.scatter(obs_customer_position[0], obs_customer_position[1], marker='.', c='w')
            n_e += 1
    # plot vehicle position 
    plt.scatter(cur_sim.current_state.current_vehicle_position[0], cur_sim.current_state.current_vehicle_position[1], marker='-', c=vehicle_color, s=60)
    plt.grid()
    plt.legend(loc='best')
    plt.title(f"t={timeline[t]} - customer locations")
    plt.show()
    print(f"number of available customers in observation:{n_e}")

interactive(children=(IntSlider(value=11, description='t', max=23), Output()), _dom_classes=('widget-interact'…

In [ ]:
@interact
# for range (min, max, step) , this gives a slider
# for drop down give a list of the options
def plot_current_obs(t=list(range(10))):
    plt.close()
    plt.scatter([2], [3+t])
    plt.xlim([0, 10])
    plt.show()

In [11]:
all_states[0].current_state

State(depot_position=array([0.22199317, 0.87073231]), current_vehicle_position=array([0.20671916, 0.91861091]), current_vehicle_capacity=40, vehicle_velocity=10, customer_positions=array([[0.48841119, 0.61174386],
       [0.76590786, 0.51841799],
       [0.2968005 , 0.18772123],
       [0.08074127, 0.7384403 ],
       [0.44130922, 0.15830987],
       [0.87993703, 0.27408646],
       [0.41423502, 0.29607993],
       [0.62878791, 0.57983781],
       [0.5999292 , 0.26581912],
       [0.28468588, 0.25358821],
       [0.32756395, 0.1441643 ],
       [0.16561286, 0.96393053],
       [0.96022672, 0.18841466],
       [0.02430656, 0.20455555],
       [0.69984361, 0.77951459],
       [0.02293309, 0.57766286],
       [0.00164217, 0.51547261],
       [0.63979518, 0.9856244 ],
       [0.2590976 , 0.80249689],
       [0.87048309, 0.92274961]]), customer_demands=array([0.02214213, 4.69488372, 9.81468738, 3.98944804, 8.13732478,
       5.46456498, 7.70854087, 4.84931075, 0.29111564, 0.86525688,
      